# News Scraping Tool for Scraping Multiple Media Websites 

#### By Joyce Jiang | Code by Joyce

Declare: My code is adapted from holwech's personal blog "Automatic news scraping with Python, Newspaper and Feedparser", you can check it out at https://holwech.github.io/blog/Automatic-news-scraper/. The author also provided a Github download, you can check it out at https://github.com/holwech/NewsScraper. This automatic scraping tool uses Newspaper library to scraping multiple news websites at the same time. You can find the documentation of Newspaper library at https://github.com/codelucas/newspaper. 

Thanks holwech for sharing a Python code. I modified this script by adding other elements, such as keywords, summary, and authors to collect from news articles with the help of Newspaper's documentation. I also developed a method to tranfer newspaper's JSON export file to a dataframe. This script is published for study and research exploration purpose only, and it would not be used for any commercial purpose. 

I used an example of Ugandan Newspaper Json file to test hotwech's method. I will also share this file in my Github Repo. 

In [1]:
import feedparser as fp
import json
import newspaper
from newspaper import Article
from time import mktime
from datetime import datetime

### Set the limit for number of articles to download

In [18]:

LIMIT = 30

data = {}
data['newspapers'] = {}

In [19]:
with open('NewsPapers_Uganda.json') as data_file:
    companies = json.load(data_file)

### Iterate through each news company

In [20]:
count = 1

# Iterate through each news company
for company, value in companies.items():
    # If a RSS link is provided in the JSON file, this will be the first choice.
    # Reason for this is that, RSS feeds often give more consistent and correct data.
    # If you do not want to scrape from the RSS-feed, just leave the RSS attr empty in the JSON file.
    if 'rss' in value:
        d = fp.parse(value['rss'])
        print("Downloading articles from ", company)
        newsPaper = {
            "rss": value['rss'],
            "link": value['link'],
            "articles": []
        }
        for entry in d.entries:
            # Check if publish date is provided, if no the article is skipped.
            # This is done to keep consistency in the data and to keep the script from crashing.
            if hasattr(entry, 'published'):
                if count > LIMIT:
                    break
                article = {}
                article['link'] = entry.link
                date = entry.published_parsed
                article['published'] = datetime.fromtimestamp(mktime(date)).isoformat()
                try:
                    content = Article(entry.link)
                    content.download()
                    content.parse()
                except Exception as e:
                    # If the download for some reason fails (ex. 404) the script will continue downloading
                    # the next article.
                    print(e)
                    print("continuing...")
                    continue
                article['title'] = content.title
                article['author']=content.authors
#                article['keywords']=content.keywords
 #               article['summary']=content.summary
                article['text'] = content.text
                newsPaper['articles'].append(article)
                print(count, "articles downloaded from", company, ", url: ", entry.link)
                count = count + 1
                
    else:
        # This is the fallback method if a RSS-feed link is not provided.
        # It uses the python newspaper library to extract articles
        print("Building site for ", company)
        paper = newspaper.build(value['link'], memoize_articles=False)
        newsPaper = {
            "link": value['link'],
            "articles": []
        }
        noneTypeCount = 0
        for content in paper.articles:
            if count > LIMIT:
                break
            try:
                content.download()
                content.parse()
            except Exception as e:
                print(e)
                print("continuing...")
                continue
            # Again, for consistency, if there is no found publish date the article will be skipped.
            # After 10 downloaded articles from the same newspaper without publish date, the company will be skipped.
            if content.publish_date is None:
                print(count, " Article has date of type None...")
                noneTypeCount = noneTypeCount + 1
                if noneTypeCount > 10:
                    print("Too many noneType dates, aborting...")
                    noneTypeCount = 0
                    break
                count = count + 1
                continue
            article = {}
            article['title'] = content.title
            article['author']=content.authors
#            article['keywords']=content.keywords
 #           article['summary']=content.summary
            article['text'] = content.text
            article['link'] = content.url
            article['published'] = content.publish_date.isoformat()
            newsPaper['articles'].append(article)
            print(count, "articles downloaded from", company, " using newspaper, url: ", content.url)
            count = count + 1
            noneTypeCount = 0
            
    count = 1
    data['newspapers'][company] = newsPaper

Building site for  Daily Monitor
1  Article has date of type None...
2  Article has date of type None...
3  Article has date of type None...
4  Article has date of type None...
5  Article has date of type None...
6  Article has date of type None...
7 articles downloaded from Daily Monitor  using newspaper, url:  https://www.monitor.co.ug/News/National/Report-projects-fall-tourism-fortunes--/688334-5577420-10te9ge/index.html
8 articles downloaded from Daily Monitor  using newspaper, url:  https://www.monitor.co.ug/News/National/Politics-alliances-against-Museveni-past-elections/688334-5577370-3m3cs4/index.html
9 articles downloaded from Daily Monitor  using newspaper, url:  https://www.monitor.co.ug/News/National/688334-5577396-mm2s7nz/index.html
10 articles downloaded from Daily Monitor  using newspaper, url:  https://www.monitor.co.ug/News/National/Top-aviation-bosses-under-investigation-minister/688334-5577390-fw7itfz/index.html
11  Article has date of type None...
12  Article has da

2 articles downloaded from CHIMP REPORTS  using newspaper, url:  https://chimpreports.com/uganda-must-strengthen-its-institutions-to-realize-socio-economic-transformation/
3 articles downloaded from CHIMP REPORTS  using newspaper, url:  https://chimpreports.com/mother-of-triplets-camps-at-district-officers-as-husband-flees/
4 articles downloaded from CHIMP REPORTS  using newspaper, url:  https://chimpreports.com/eacj-court-concludes-its-inception-online-sessions-successfully/
5 articles downloaded from CHIMP REPORTS  using newspaper, url:  https://chimpreports.com/private-school-owners-ask-govt-to-create-education-recovery-fund/
6 articles downloaded from CHIMP REPORTS  using newspaper, url:  https://chimpreports.com/masaza-cup-gomba-seals-17th-signing/
7 articles downloaded from CHIMP REPORTS  using newspaper, url:  https://chimpreports.com/4-arrested-for-killing-buffalo-calf/
8 articles downloaded from CHIMP REPORTS  using newspaper, url:  https://chimpreports.com/court-orders-taxi-b

1  Article has date of type None...
2  Article has date of type None...
3  Article has date of type None...
4  Article has date of type None...
5  Article has date of type None...
6  Article has date of type None...
7  Article has date of type None...
8  Article has date of type None...
9  Article has date of type None...
10  Article has date of type None...
11  Article has date of type None...
Too many noneType dates, aborting...
Building site for  THE INDEPENDENT
1 articles downloaded from THE INDEPENDENT  using newspaper, url:  https://www.independent.co.ug/german-virus-hunters-track-down-corona-outbreaks/
2 articles downloaded from THE INDEPENDENT  using newspaper, url:  https://www.independent.co.ug/a-world-redrawn-nobel-winner-deaton-warns-virus-could-worsen-inequality/
3 articles downloaded from THE INDEPENDENT  using newspaper, url:  https://www.independent.co.ug/open-letter-to-president-yoweri-museveni-decision-not-to-reopen-schools-may-do-more-harm-than-good/
4 articles downl

19 articles downloaded from BUKEDDE  using newspaper, url:  https://www.bukedde.co.ug/news/1520710/ebiri-mu-kiraamo-kya-kasirye-ggwanga
20 articles downloaded from BUKEDDE  using newspaper, url:  https://www.bukedde.co.ug/amawulire/1520743/ebyewuunyisa-ku-kasirye-gwanga-obadde-tomanyi
21 articles downloaded from BUKEDDE  using newspaper, url:  https://www.bukedde.co.ug/news/1520669/ekyagaanyi-kasirye-ggwanga-okuziikibwa-ku-kiggya-kya-ffamire
22 articles downloaded from BUKEDDE  using newspaper, url:  https://www.bukedde.co.ug/ssenga/1520933/engeri-abafumbo-gye-babadde-bakukuta-ku-ssimu-mu-kalantiini
23 articles downloaded from BUKEDDE  using newspaper, url:  https://www.bukedde.co.ug/ssenga/1520931/kikyamu-abaana-bo-okusiibanga-ku-muliraano-olw-obutagula-byakulya-waka
24 articles downloaded from BUKEDDE  using newspaper, url:  https://www.bukedde.co.ug/ssenga/1520935/njagala-mwami-alina-empisa
Article `download()` failed with 500 Server Error: Internal Server Error for url: https://www

19 articles downloaded from OBSERVER  using newspaper, url:  https://observer.ug/appnews/59403-unrest-as-banks-deny-govt-agencies-loans
20 articles downloaded from OBSERVER  using newspaper, url:  https://observer.ug/life/59383-mother-kevin-embarks-on-long-journey-to-beatification
21 articles downloaded from OBSERVER  using newspaper, url:  https://observer.ug/appnews/59189-bou-fighting-for-most-critical-asset-reputation
22 articles downloaded from OBSERVER  using newspaper, url:  https://observer.ug/businessnews/65254-mtn-pays-shs-372bn-for-operating-licence-for-next-12-years
23 articles downloaded from OBSERVER  using newspaper, url:  https://observer.ug/businessnews/65203-standard-bank-predicts-tough-future-for-uganda-s-economy
24 articles downloaded from OBSERVER  using newspaper, url:  https://observer.ug/businessnews/65202-business-leaders-call-for-more-digital-platforms
25 articles downloaded from OBSERVER  using newspaper, url:  https://observer.ug/businessnews/65226-mtn-discus

10 articles downloaded from PC TECH  using newspaper, url:  https://pctechmag.com/2020/06/airtel-10-years-in-uganda/
11 articles downloaded from PC TECH  using newspaper, url:  https://pctechmag.com/2020/06/appointing-nira-boss-with-it-backgroun/
12 articles downloaded from PC TECH  using newspaper, url:  https://pctechmag.com/2020/06/eversend-donation-feature/
13 articles downloaded from PC TECH  using newspaper, url:  https://pctechmag.com/2020/06/undp-offer-govt-ict-equipment-to-combat-covid19/
14 articles downloaded from PC TECH  using newspaper, url:  https://pctechmag.com/2020/06/hiil-uganda-kampala-legal-hackathon/
15 articles downloaded from PC TECH  using newspaper, url:  https://pctechmag.com/2020/06/u-s-huawei-ban-is-about-5g-leadership-race/
16 articles downloaded from PC TECH  using newspaper, url:  https://pctechmag.com/2020/06/mtn-clarifies-on-ruling-to-pay-ugx-24-billion-in-taxes-to-ura/
17 articles downloaded from PC TECH  using newspaper, url:  https://pctechmag.com/2

10 articles downloaded from BIG EYE  using newspaper, url:  https://bigeye.ug/sheilah-gashumba-states-the-things-she-loves-about-gods-plan/
11 articles downloaded from BIG EYE  using newspaper, url:  https://bigeye.ug/all-you-need-to-know-about-rising-star-revboy/
12 articles downloaded from BIG EYE  using newspaper, url:  https://bigeye.ug/chamili-congratulates-spice-daina-upon-receiving-youtube-award/
13 articles downloaded from BIG EYE  using newspaper, url:  https://bigeye.ug/mpaka-records-dre-cali-drops-new-song-ebisooka-ne-bisembayo/
14 articles downloaded from BIG EYE  using newspaper, url:  https://bigeye.ug/video-iry-tina-releases-drops-new-hold-me/
15 articles downloaded from BIG EYE  using newspaper, url:  https://bigeye.ug/video-alert-vivian-tendo-praises-imaginary-lover-in-mu-kati/
16 articles downloaded from BIG EYE  using newspaper, url:  https://bigeye.ug/video-you-are-a-legend-in-this-game-rude-boy-tells-chameleone/
17 articles downloaded from BIG EYE  using newspaper,

13 articles downloaded from SQOOP  using newspaper, url:  https://www.sqoop.co.ug/202006/four-one-one/ratata-its-zex-bilangilangi.html
14 articles downloaded from SQOOP  using newspaper, url:  https://www.sqoop.co.ug/202006/four-one-one/eddy-kenzo-spice-diana-eat-big.html
15 articles downloaded from SQOOP  using newspaper, url:  https://www.sqoop.co.ug/202006/four-one-one/cinemas-open-then-asked-to-close-again.html
16 articles downloaded from SQOOP  using newspaper, url:  https://www.sqoop.co.ug/202006/news/bad-blacks-advert-was-voluntary-says-government.html
17 articles downloaded from SQOOP  using newspaper, url:  https://www.sqoop.co.ug/202006/news/bad-black-sues-govt-over-failure-to-pay-for-covid-19-advert.html
18 articles downloaded from SQOOP  using newspaper, url:  https://www.sqoop.co.ug/202006/news/sacked-sanyu-fm-employees-blame-managers-over-strike.html
19 articles downloaded from SQOOP  using newspaper, url:  https://www.sqoop.co.ug/202006/features/celebrity-profiles/i-migh

#### *The limit amount of articles that are allowed to scrape depends on each website. My experience is that the limit number varies from 25 to 100 articles. With RSS Feed URL you will be able to scrape more articles. 

In [7]:
try:
    with open('scraped_articles.json', 'w') as outfile:
        json.dump(data, outfile)
except Exception as e: print(e)

### Convert json file to dataframe

In [8]:
import csv 
import pandas as pd
import pprint
import pprintpp

In [9]:
with open('scraped_articles.json') as json_file: 
    data = json.load(json_file) 

In [10]:
pprintpp.pprint(data)

{
    'newspapers': {
        'bbc': {
            'articles': [
                {
                    'author': ['Mark Piesing'],
                    'link': 'https://www.bbc.com/future/article/20200615-the-record-breaking-jet-which-still-haunts-a-country',
                    'published': '2020-06-15T00:00:00',
                    'text': 'In the early years of the Cold War, Canada decided to design and build the most advanced fighter aircraft in the world.\n\nCanada is well known for its rugged bush planes, capable of rough landings and hair-raising take-offs in the wilderness. From the late 1930s, the North American country had also started to manufacture British-designed planes for the Allied war effort. Many of these planes were iconic wartime designs like the Hawker Hurricane fighter and Avro Lancaster bomber.\n\nAmbitious Canadian politicians and engineers weren’t satisfied with this. They decided to forge a world-leading aircraft manufacturing industry out of the factories and

                    'text': 'The COVID Tracking Project reported on Monday that COVID-19 deaths continue to fall in the United States but also noted in a series of tweets the number of cases is up slightly over the past seven days and that they are watching closely five states where current hospitalizations have increased over the past two weeks.\n\nThese are the five states we’re watching most closely. pic.twitter.com/cHnFPHUg9M — The COVID Tracking Project (@COVID19Tracking) June 15, 2020\n\nStates reported 18,521 positives today, up from 17,123 last Monday. pic.twitter.com/XcIQed7QbD — The COVID Tracking Project (@COVID19Tracking) June 15, 2020\n\nNate Silver added his own analysis of the recent COVID-19 data:\n\nThere continues to be this ebb-and-flow where people alternatively become too optimistic or too pessimistic in ways that are somewhat detached from the data. Certainly, the death numbers have been better, lately. 7-day average now at ~700, down from ~2,000 at the peak. — Na

                    'text': "The Supreme Court’s (SCOTUS) reinterpretation of a federal prohibition against employment discrimination based on sex — which now includes sexual orientation and “gender identity” — will “create a tsunami of new litigation” against religious organizations, explained Carrie Severino, president of the Judicial Crisis Network, offering her remarks on SiriusXM’s Breitbart News Tonight with host Rebecca Mansour.\n\nThe Supreme Court’s legal redefinition of “sex” to include sexual orientation and “gender identity” opens the door for further left-wing lawfare against religious organizations, Severino noted.\n\nSeverino said, “The Supreme Court left a lot of really important issues open, like, how do you balance this with religious freedom? How do you balance it with freedom of speech? If you’ve got a law, for example, saying that using someone’s preferred pronoun is mandatory — or you can be fined [for non-compliance], how do we balance that with some of these oth

                    'text': 'In an interview with Politico on Sunday, Chicago Democrat Congressman Bobby Rush attacked Chicago law enforcement.\n\nLong-time activist and career politician Bobby L. Rush unleashed a scathing review of law enforcement officers on Sunday, comparing them to the Ku Klux Klan. “The number-one cause that prevents police accountability, that promotes police corruption, that protects police lawlessness, is a culprit called the Fraternal Order of Police,” he told Politico, calling the two organizations “kissing, hugging and law-breaking cousins”:\n\nThey’re the organized guardians of continuous police lawlessness, of police murder and police brutality. The Chicago Fraternal Order of Police is the most rabid, racist body of criminal lawlessness by police in the land. It stands shoulder to shoulder with the Ku Klux Klan then and the Ku Klux Klan now.\n\nRush’s words were part of his response to a leaked video showing police at leisure in his office on the weekend o

                    'link': 'http://edition.cnn.com/2020/06/11/health/regeneron-covid-19-antibody-trial-starts/index.html',
                    'published': '2020-06-11T00:00:00',
                    'text': '(CNN) A medicine that may treat and prevent Covid-19 is now being tested in patients in multiple sites around the United States, according to an announcement Thursday from Regeneron Pharmaceuticals Inc.\n\nIt is the first trial of a Covid-19 antibody cocktail in the United States. If successful, Regeneron hopes it could be available by the fall.\n\nThe clinical trial started Wednesday. Regeneron said its antibody cocktail will be tested in four separate study populations: people who are hospitalized with Covid-19; people who have symptoms for the disease, but are not hospitalized; people who are healthy but are at a high risk for getting sick; and healthy people who have come into close contact with a person who is sick.\n\n"We have created a unique anti-viral antibody cocktail wi

...

                    'text': 'So Carlos Machado, a senior scientist with Brazil’s prestigious Oswaldo Cruz Foundation, wanted the language to be strong. At the request of Rio officials, his team was assembling a list of recommendations. He needed to make clear what would happen if they didn’t immediately impose a complete lockdown.\n\n“It would result,” the team warned in the early May report, “in a human catastrophe of unimaginable proportions.”\n\nAD\n\nBut the officials never instituted a lockdown. Cases and deaths soared. People stopped isolating, choosing instead to pack beach boardwalks on weekends. And the warning turned out to be just one more exit ramp that Brazil declined to take on its way toward becoming the second-most disease-ravaged country in the world.\n\nLatin America’s largest country has so far registered more than 888,000 coronavirus cases and nearly 44,000 deaths, second on both counts only to the United States. But while other countries have been through steep cur

                    'text': 'The problem for Netanyahu and his allies is that much of the world opposes their plans. In Washington, Democrats and even some Republicans are against it, with Trump and Netanyahu’s close embrace widening a partisan split over support for Israel.\n\nAD\n\nAD\n\nFor the Palestinian leadership, annexation is a non-starter, and they have reacted angrily, suspending coordination with Israel on a host of day-to-day matters. “This is an issue in which we cannot be silent on,” Palestinian Prime Minister Mohammad Shtayyeh told reporters last week. “Annexation is an existential threat for our future.” (Many ordinary Palestinians, though, may argue that it would simply be further confirmation of a long-standing status quo in which their freedoms and political rights are subordinate to — and often curtailed by — the imperatives of Israeli security.)\n\nThough Netanyahu and Trump pay lip service to the future viability of an independent Palestinian state, no serious ex

                    'text': 'Whelan, arrested on Dec. 28, 2018, has said he thought the flash drive that he received from an acquaintance contained holiday photos. He said Monday that he plans to appeal the court’s decision.\n\nAD\n\nNow that Whelan has been convicted, speculation is rife about a possible prisoner exchange with the United States. Zherebenkov said Monday that “Paul expected this decision because even when he was detained, he was told [by Russian security service agents] that he would be exchanged.”\n\nAD\n\nWithout revealing his source, Zherebenkov said he was told Konstantin Yaroshenko, a pilot who was arrested in 2010 for conspiracy to smuggle cocaine into the United States, and Viktor Bout, a gun runner who inspired the 2005 Hollywood film “Lord of War,” are the people the Kremlin is focused on as possible trades for Whelan’s release.\n\n“I heard talk that, why should we waste time on the appeal if we can just go ahead with the exchange?” Zherebenkov said. “I can’t g

                    'text': '\n\nHail wrecked the siding of homes in Calgary on Saturday. (Ben Chernesky/Twitter @Zombery360)\n\nA vicious hailstorm pounded Calgary, Canada, on Saturday, turning a summer day into a virtual winter wonderland as chunks of ice the size of tennis balls ricocheted out of the sky. Wind-driven hailstones shattered windows and windshields, peeled the siding off homes and businesses, and coated the ground several inches thick, making it a monochromatic icy landscape.\n\n[Towering corkscrew-shaped storms swirled over Alberta last June]\n\nTrees were stripped bare in some areas where heavy hail shredded vegetation. Elsewhere, reports of funnel clouds accompanied the ominous storm as it churned through the city of 1.4 million.\n\nThe same storms produced serious flooding, inundating streets and leaving numerous vehicles up to their canopies in water. The region had been under a severe thunderstorm watch for much of the afternoon.\n\nCalgary International Airport f

                    'title': 'As control measures lift, the coronavirus pandemic continues to grow. Here are the global hot spots.',
                },
            ],
            'link': 'https://www.washingtonpost.com/',
            'rss': 'http://feeds.washingtonpost.com/rss/world',
        },
    },
}


In [12]:
df = pd.read_json ('scraped_articles.json')
df

,newspapers
bbc,{'link': 'https://www.bbc.co.uk/search?q=covid...
breitbart,"{'link': 'http://www.breitbart.com/', 'article..."
cnn,"{'link': 'http://edition.cnn.com/', 'articles'..."
foxnews,"{'link': 'http://www.foxnews.com/', 'articles'..."
infowars,"{'link': 'https://www.infowars.com/', 'article..."
nbcnews,"{'link': 'http://www.nbcnews.com/', 'articles'..."
theguardian,"{'rss': 'https://www.theguardian.com/uk/rss', ..."
theonion,"{'link': 'http://www.theonion.com/', 'articles..."
washingtonpost,{'rss': 'http://feeds.washingtonpost.com/rss/w...


In [13]:
#pprintpp.pprint(data['newspapers']['cnn']['articles'][3]['author'])

In [14]:
outlets=[]
titles=[]
authors=[]
texts=[]
links=[]
publisheds=[]

def get_list(name):
    for x in range(len(data['newspapers'][name]['articles'])):
        outlets.append(name)
        titles.append(data['newspapers'][name]['articles'][x]['title'])
        authors.append(data['newspapers'][name]['articles'][x]['author'])
        texts.append(data['newspapers'][name]['articles'][x]['text'])
        links.append(data['newspapers'][name]['articles'][x]['link'])
        publisheds.append(data['newspapers'][name]['articles'][x]['published'])
    

get_list('cnn')
get_list('bbc')
get_list('theguardian')
get_list('breitbart')
get_list('infowars')
get_list('foxnews')
get_list('nbcnews')
get_list('washingtonpost')
get_list('theonion')

get_list('Daily Monitor')
get_list('New Vision')
get_list('CHIMP REPORTS')
get_list('NILE POST')
get_list('UGANDA RADIO NETWORK')
get_list('THE INDEPENDENT')
get_list('BUKEDDE')
get_list('RED PEPPER')
get_list('OBSERVER')
get_list('PML DAILY')
get_list('SOFT POWER NEWS')
get_list('PC TECH')
get_list('TECH JAJA')
get_list('BIG EYE')
get_list('HIPIPO')
get_list('SQOOP')
get_list('HOWWEBIZ')
get_list('KAWOWO SPORTS')
get_list('NTV UGANDA')
get_list('NBS TV')

In [15]:
get_list('cnn')
get_list('bbc')
get_list('foxnews')
#get_list('NILE POST')
#get_list('UGANDA RADIO NETWORK')
#get_list('THE INDEPENDENT ')
#get_list('BUKEDDE')
#get_list('RED PEPPER')
#get_list('OBSERVER')
#get_list('PML DAILY')
#get_list('SOFT POWER NEWS')
#get_list('PC TECH')
#get_list('TECH JAJA')
#get_list('BIG EYE')
#get_list('HIPIPO')
#get_list('SQOOP')
#get_list('HOWWEBIZ')
#get_list('KAWOWO SPORTS')

In [16]:
dic={
    'newspaper':outlets,
    'title':titles,
    'authors':authors,
    'text':texts,
    'link':links,
    'published':publisheds
}

df2 = pd.DataFrame(dic, columns = ['newspaper','title', 'authors','text', 'link','published'])
df2

,newspaper,title,authors,text,link,published
0,cnn,US surgeons successfully perform double lung t...,[Jacqueline Howard],(CNN) A young woman in the United States whose...,http://edition.cnn.com/2020/06/11/health/lung-...,2020-06-11T00:00:00
1,cnn,US human trials begin for first antibody cockt...,[Jen Christensen],(CNN) A medicine that may treat and prevent Co...,http://edition.cnn.com/2020/06/11/health/regen...,2020-06-11T00:00:00
2,cnn,Grocery stores and universities should reopen ...,[Jen Christensen],"(CNN) New research suggests grocery stores, ba...",http://edition.cnn.com/2020/06/10/health/groce...,2020-06-10T00:00:00
3,cnn,Asymptomatic coronavirus spread: WHO clarifies...,[Jacqueline Howard],(CNN) The World Health Organization tried on T...,http://edition.cnn.com/2020/06/09/health/who-c...,2020-06-09T00:00:00
4,cnn,Dexamethasone and Covid: Steroid reduces risk ...,[Jacqueline Howard],(CNN) The widely available steroid drug dexame...,http://edition.cnn.com/2020/06/16/health/dexam...,2020-06-16T00:00:00
5,cnn,"Model projects 200,000 people in the US could ...",[Madeline Holcombe],(CNN) The United States could see more than 20...,http://edition.cnn.com/2020/06/16/health/us-co...,2020-06-16T00:00:00
6,cnn,Children with ADHD can now be prescribed a vid...,"[Naomi Thomas, Amy Woodyatt]",(CNN) The first video game-based treatment for...,http://edition.cnn.com/2020/06/16/health/adhd-...,2020-06-16T00:00:00
7,cnn,Try this total-body summer workout inspired by...,[Stephanie Mansour],(CNN) Whether you're a child or an adult who's...,http://edition.cnn.com/2020/06/16/health/playg...,2020-06-16T00:00:00
8,bbc,The record-breaking jet which still haunts a c...,[Mark Piesing],"In the early years of the Cold War, Canada dec...",https://www.bbc.com/future/article/20200615-th...,2020-06-15T00:00:00


for x in range(len(data['newspapers']['cnn']['articles'])):
    outlets.append('CNN')
    titles.append(data['newspapers']['cnn']['articles'][x]['title'])
    texts.append(data['newspapers']['cnn']['articles'][x]['text'])
    links.append(data['newspapers']['cnn']['articles'][x]['link'])
    publisheds.append(data['newspapers']['cnn']['articles'][x]['published'])

In [17]:
df2.to_excel('Output/Newspaper_update.xlsx')